In [1]:
import pandas as pd  
import numpy as np   
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import  Dropout, Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, f1_score

In [16]:
train = pd.read_csv("/content/train.csv")
valid = pd.read_csv("/content/validation.csv")
test = pd.read_csv("/content/test.csv")

In [ ]:
train.head()

,Age,Educational_Level,Income,Adults,Children,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate,is_female,Is_Latino,Is_caucasian,Is_African American,Is_Asian American,East,North,South,West,Airport Hotels,City Hotel,Resort,Deposit_type,Agent,Online,Direct,Total_People,Rooms_Needed,Total_Days_Staying,Total_Room_Rate,Days_since_Booking,WeekDay,Staying_in_Weekend,Checkin_Year,Checkin_Month,Reservation_Status
0,18.0,1.0,1.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,10.0,129.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,116.10,224.0,1.0,0.0,2016.0,5.0,0
1,18.0,2.0,4.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,25.0,130.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,97.50,168.0,1.0,0.0,2016.0,4.0,0
2,18.0,3.0,1.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,25.0,114.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,85.50,160.0,1.0,0.0,2016.0,4.0,0
3,18.0,3.0,3.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,181.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,181.00,124.0,4.0,0.0,2016.0,3.0,0
4,18.0,3.0,3.0,2.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,25.0,205.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,153.75,190.0,6.0,1.0,2016.0,7.0,0


In [ ]:
train.describe()

,Age,Educational_Level,Income,Adults,Children,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Required_Car_Parking,Use_Promotion,Discount_Rate,Room_Rate,is_female,Is_Latino,Is_caucasian,Is_African American,Is_Asian American,East,North,South,West,Airport Hotels,City Hotel,Resort,Deposit_type,Agent,Online,Direct,Total_People,Rooms_Needed,Total_Days_Staying,Total_Room_Rate,Days_since_Booking,WeekDay,Staying_in_Weekend,Checkin_Year,Checkin_Month,Reservation_Status
count,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000,63720.000000
mean,43.909660,2.596388,2.291329,2.334338,1.752136,0.354478,1.845223,0.329283,0.110572,0.718168,0.756538,12.150706,175.223785,0.501297,0.249500,0.249562,0.255763,0.245175,0.195699,0.210259,0.398148,0.195894,0.333628,0.329655,0.336717,1.573749,0.141716,0.564072,0.294213,4.086475,1.158315,1.825276,323.776490,109.402408,3.984906,0.534857,2015.811357,6.164318,1.000000
std,14.561583,0.916019,0.927993,1.077412,0.653896,0.515187,0.728839,0.423747,0.282352,0.405027,0.404418,10.621907,43.434664,0.450371,0.389949,0.390245,0.393242,0.387555,0.356695,0.366564,0.441168,0.357411,0.425087,0.423412,0.424715,0.772543,0.314747,0.446510,0.410091,1.265060,0.340699,0.960620,229.839207,69.922872,1.818991,0.454469,0.569493,2.954645,0.816503
min,18.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,60.000000,-4.000000,1.000000,0.000000,2014.000000,1.000000,0.000000
25%,31.367196,2.000000,1.528489,1.665715,1.000000,0.000000,1.000000,0.000000,0.000000,0.364160,0.555299,3.720915,137.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000,1.000000,1.000000,163.099654,50.000000,2.486891,0.000000,2015.266894,3.964892,0.000000
50%,44.000000,2.999684,2.052790,2.000000,1.946175,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,10.000000,175.290137,0.501703,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120946,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.727207,0.000000,4.000000,1.000000,1.503479,244.000000,105.431044,4.000000,0.638076,2016.000000,6.020210,1.000000
75%,56.336245,3.000000,3.000000,3.000000,2.000000,0.786994,2.403695,0.819300,0.000000,1.000000,1.000000,20.000000,213.971229,1.000000,0.498439,0.497531,0.528172,0.473417,0.201447,0.289761,1.000000,0.198096,0.833619,0.818465,0.836946,2.049456,0.000000,1.000000,0.688092,5.000000,1.000000,2.495425,417.152548,165.000000,5.409526,1.000000,2016.000000,8.086343,2.000000
max,70.000000,4.000000,4.000000,5.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,40.000000,250.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,8.000000,2.000000,4.000000,1992.000000,708.000000,7.000000,1.000000,2017.000000,12.000000,2.000000


In [10]:
train.Reservation_Status.value_counts()

0    21240
1     4134
2     2125
Name: Reservation_Status, dtype: int64

In [ ]:
valid.Reservation_Status.value_counts()

0    1610
1     741
2     398
Name: Reservation_Status, dtype: int64

In [17]:
drop_cols = ['Is_Latino', 'East', 'Expected_checkin','Expected_checkout', 'Booking_date', 'Total_People', 'Resort', 'Agent', 'Use_Promotion', 'Room_Rate', 'Rooms_Needed']
train.drop(drop_cols, axis=1, inplace=True)
valid.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [18]:
X_train = train.drop('Reservation_Status',axis=1)
y_train = train.Reservation_Status
X_valid = valid.drop('Reservation_Status',axis=1)
y_valid = valid.Reservation_Status

columns = X_train.columns

In [19]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(test)

In [21]:
sm = SMOTE(random_state=1, sampling_strategy={0:21250,1:10000,2:7000}) 

In [22]:
X_train, y_train = sm.fit_sample(X_train, y_train)
X_train = pd.DataFrame(X_train, columns=columns)
y_train = pd.DataFrame(y_train, columns=['Reservation_Status'])

In [ ]:
# X_valid, y_valid = sm.fit_sample(X_valid, y_valid)
# X_valid = pd.DataFrame(X_valid, columns=columns)
# y_valid = pd.DataFrame(y_valid, columns=['Reservation_Status'])

In [23]:
y_train.value_counts()

Reservation_Status
0                     21250
1                     10000
2                      7000
dtype: int64

In [ ]:
y_valid.value_counts()

0    1610
1     741
2     398
Name: Reservation_Status, dtype: int64

In [ ]:
X_train.shape

(63720, 38)

In [ ]:
X_valid.shape

(2749, 38)

In [ ]:
X_test.shape

(4318, 38)

In [ ]:
# X_train["Reservation_Status"] = y_train
# X_valid["Reservation_Status"] = y_valid

In [ ]:
# X_train.to_csv('train.csv', index=False)      
# files.download('train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# X_valid.to_csv('validation.csv', index=False)      
# files.download('validation.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
#model building
model = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=3, activation='softmax'),
])

In [32]:
model.compile(optimizer='Adam', metrics = ['accuracy'], loss='sparse_categorical_crossentropy', )
history = model.fit(x=X_train, y=y_train, batch_size=128, epochs=120, verbose=1, validation_data=(X_valid, y_valid),)

Epoch 1/120
299/299 [==============================] - 1s 3ms/step - loss: 0.9932 - accuracy: 0.5318 - val_loss: 0.9972 - val_accuracy: 0.5406
Epoch 2/120
299/299 [==============================] - 1s 2ms/step - loss: 0.9384 - accuracy: 0.5583 - val_loss: 0.9895 - val_accuracy: 0.5558
Epoch 3/120
299/299 [==============================] - 1s 2ms/step - loss: 0.9219 - accuracy: 0.5693 - val_loss: 0.9945 - val_accuracy: 0.5515
Epoch 4/120
299/299 [==============================] - 1s 2ms/step - loss: 0.9158 - accuracy: 0.5718 - val_loss: 1.0103 - val_accuracy: 0.5406
Epoch 5/120
299/299 [==============================] - 1s 2ms/step - loss: 0.8977 - accuracy: 0.5837 - val_loss: 1.0287 - val_accuracy: 0.4878
Epoch 6/120
299/299 [==============================] - 1s 2ms/step - loss: 0.8924 - accuracy: 0.5816 - val_loss: 1.0446 - val_accuracy: 0.4744
Epoch 7/120
299/299 [==============================] - 1s 2ms/step - loss: 0.8772 - accuracy: 0.5949 - val_loss: 1.0462 - val_accuracy: 0.5035

In [33]:
pred_valid = model.predict(X_valid)
pred_valid = np.argmax(pred_valid, axis=-1)
f1_score(y_valid, pred_valid, average='macro')

0.34901812599422083

In [ ]:
out = pd.read_csv("/content/Hotel-A-test.csv")
out = pd.DataFrame(out['Reservation-id'])
out['Reservation_status'] = np.argmax(model.predict(X_test), axis=-1)+1
out.to_csv('submit.csv', index=False)      
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>